In [1]:
import cv2, io, time, imutils, datetime, os, traceback, grpc
from imutils.video import VideoStream
import opencv_jupyter_ui as jcv2
import numpy as np

from tensorflow import make_tensor_proto, make_ndarray
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

2022-08-12 11:25:58.092827: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:
2022-08-12 11:25:58.092850: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
MODEL_NAME = "face-detection"
INPUT_NAME = "data"
RTSP_URL = "rtsp://localhost:8554/server"
GRPC_URL = "localhost:9000"
BATCH_SIZE = 1

WIDTH = 300
HEIGHT = 300

In [3]:
channel = grpc.insecure_channel(GRPC_URL)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
print("* GRPC Channel succesfully opened")

try:
    rtsp_stream = VideoStream(RTSP_URL).start()
    print("* RTSP Stream succesfully opened")
except Exception as e:
    print("* RTSP Stream is unavailable, please close any other notebooks that might be accessing the stream first")

* GRPC Channel succesfully opened
* RTSP Stream succesfully opened


In [4]:
def load_image(img):
    #img = np.frombuffer(img, dtype=np.uint8)
    #img = cv2.imdecode(img, cv2.IMREAD_COLOR)  # BGR format
    img = cv2.resize(img, (WIDTH, HEIGHT))
    img = img.transpose(2,0,1).reshape(1,3,HEIGHT, WIDTH)
    img = img.astype('float32')
    return img 

In [5]:
def draw_boxes(output, img):
     for i in range(0, 200* BATCH_SIZE - 1):  # there is returned 200 detections for each image in the batch
            detection = output[:,:,i,:]
            # each detection has shape 1,1,7 where last dimension represent:
            # image_id - ID of the image in the batch
            # label - predicted class ID
            # conf - confidence for the predicted class
            # (x_min, y_min) - coordinates of the top left bounding box corner
            #(x_max, y_max) - coordinates of the bottom right bounding box corner.
            if detection[0,0,2] > 0.5:  # ignore detections for imageid != y and confidence <0.5
                x_min = int(detection[0,0,3] * 640)
                y_min = int(detection[0,0,4] * 480)
                x_max = int(detection[0,0,5] * 640)
                y_max = int(detection[0,0,6] * 480)

                img = cv2.rectangle(img,(x_min,y_min),(x_max,y_max),(0, 255, 0) ,1)

In [6]:
def run_inference(img):
    request = predict_pb2.PredictRequest()
    request.model_spec.name = MODEL_NAME
    request.inputs["data"].CopyFrom(make_tensor_proto(img, shape=(img.shape)))
    result = stub.Predict(request, 10.0)
    output = make_ndarray(result.outputs["detection_out"])

    return output

In [ ]:
while True:
    try: 
        frame = rtsp_stream.read()
        try: 
            if frame is not None:
                prepared = load_image(frame)
                output = run_inference(prepared)
                #frame = cv2.flip(frame, 1)
                draw_boxes(output, frame)
                jcv2.imshow("Prediction Output", frame)
            else: 
                continue 
                
            time.sleep(1/10)
        except Exception as e:
            print("error while running inference", e)
            print(traceback.format_exc())
          
    except Exception as e:
        print("error while grabbing RTSP stream", e)

450 411 495 474
452 372 496 437
466 336 512 402
466 326 514 391
464 320 507 381
460 309 504 371
460 288 506 352
467 269 512 325
472 176 531 253
462 220 513 285
460 244 510 303
442 301 491 356
444 322 486 380
402 380 452 443
402 362 453 422
477 312 533 367
520 298 577 350
529 296 583 352
527 303 580 355
522 317 568 376
513 336 551 398
502 349 540 407
498 368 535 422
486 378 519 425
481 392 516 439
430 395 468 447
392 409 439 469
524 380 551 425
393 370 434 429
409 354 453 418
545 361 579 412
552 362 598 416
421 343 467 408
429 336 474 402
563 364 604 425
432 333 479 398
563 356 609 414
428 334 477 396
420 338 470 398
565 335 612 390
418 337 463 398
565 328 609 385
407 345 448 405
565 317 612 381
403 349 442 407
550 295 594 354
542 291 589 341
506 275 563 336
491 264 549 327
486 253 544 316
483 239 544 308
484 220 544 294
482 185 544 259
479 168 542 248
473 146 535 219
461 125 525 200
439 107 509 176
415 89 482 165
385 54 454 129
361 29 440 108
354 19 431 105
350 15 427 93
344 7 420 86
3